# Webscraping
## Portal da Queixa
### Changes may be required due to Portal da Queixa's continous updates

(c) Nickolas Lago 2021 - Rev. 1.0

### Load packages and do the initializations

In [24]:
# Load libraries
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import category_encoders as ce
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

### Importing the data from the provious webscraping

Delete the Unnamed: 0 column and make the complainID the index column of the DataFrame

In [13]:
df = pd.read_excel("complains_data.xlsx", index_col="complainID")

In [14]:
cols_to_drop = ["Unnamed: 0"]
df = df.drop(cols_to_drop, axis=1)

In [15]:
df.head()

,complainStatus,complainUser,complainTitle,complainViews,complainText,complainDate
complainID,,,,,,
59476521,Aguarda resposta,Susana,Lefties - Devolução de artigos,51,Recebi a minha encomenda no dia 19-04-2021 e i...,5 de maio 2021
58935721,Aguarda resposta,Cassia Barcelos,Lefties - Troca de peças compradas on-line,172,"Boa noite, _x000D_\nVenho mostrar o meu total ...",23 de abril 2021
58754421,Aguarda resposta,Milene,Lefties - Encomenda não entregue,59,A minha encomenda que fiz no dia 19 de Março n...,20 de abril 2021
58696921,Aguarda resposta,Mariana Ferreira,Lefties - Encomenda não entregue nem resolvem ...,38,Encomenda nr ‪90003989775‬ já deveria ter sido...,19 de abril 2021
58650521,Resolvida,Olga Santos,Lefties - Encomenda não entregue,121,Bom dia. Fiz uma encomenda online na Lefties n...,17 de abril 2021


## Working with Text

Cleaning and normalization of the raw text.

In [16]:
textRaw = df.complainText[58935721]

print(textRaw)

Boa noite, _x000D_
Venho mostrar o meu total descontentamento e frustação relativamente ao serviço prestado. _x000D_
Fiz uma encomenda de várias peças, uma delas um pack de 4 calças de fato de treino por 15,99€. Ao chegar a encomenda deparei-me que em vez do pack só tinha recebido 1 calça em vez das 4. Enviei e-mail para o apoio ao cliente a expor a situação, passado algumas horas responderam a afirmar que teriam entrado em contacto comigo várias vezes sem sucesso (o que é mentira pois confirmei o número de telefone cedido na minha conta e não tinha nenhuma chamada não atendida), pediram para retribuir a chamada. _x000D_
Contactei então por chamada telefonica, ao qual a assistente que me atendeu nunca pediu desculpa pelo sucedido mas explicou que deveria devolver a calça com uma etiqueta própria e que me seria enviado o pack completo. _x000D_
Passado dois dias recebi o pack mas desta vez 1 das 4 calças veio com vários tamanhos abaixo do pedido. _x000D_
Quero que resolvam esta situação 

In [17]:
# normalize the case

textNormalized = textRaw.lower()

print(textNormalized)

boa noite, _x000d_
venho mostrar o meu total descontentamento e frustação relativamente ao serviço prestado. _x000d_
fiz uma encomenda de várias peças, uma delas um pack de 4 calças de fato de treino por 15,99€. ao chegar a encomenda deparei-me que em vez do pack só tinha recebido 1 calça em vez das 4. enviei e-mail para o apoio ao cliente a expor a situação, passado algumas horas responderam a afirmar que teriam entrado em contacto comigo várias vezes sem sucesso (o que é mentira pois confirmei o número de telefone cedido na minha conta e não tinha nenhuma chamada não atendida), pediram para retribuir a chamada. _x000d_
contactei então por chamada telefonica, ao qual a assistente que me atendeu nunca pediu desculpa pelo sucedido mas explicou que deveria devolver a calça com uma etiqueta própria e que me seria enviado o pack completo. _x000d_
passado dois dias recebi o pack mas desta vez 1 das 4 calças veio com vários tamanhos abaixo do pedido. _x000d_
quero que resolvam esta situação 

In [18]:
charsToRemove = r'\_x000d_|\?|\.|\!|\;|\.|\"|\,|\(|\)|\&|[0-9]'

textWOPontuation = re.sub(charsToRemove,'', textNormalized,)

print(textWOPontuation)

boa noite 
venho mostrar o meu total descontentamento e frustação relativamente ao serviço prestado 
fiz uma encomenda de várias peças uma delas um pack de  calças de fato de treino por € ao chegar a encomenda deparei-me que em vez do pack só tinha recebido  calça em vez das  enviei e-mail para o apoio ao cliente a expor a situação passado algumas horas responderam a afirmar que teriam entrado em contacto comigo várias vezes sem sucesso o que é mentira pois confirmei o número de telefone cedido na minha conta e não tinha nenhuma chamada não atendida pediram para retribuir a chamada 
contactei então por chamada telefonica ao qual a assistente que me atendeu nunca pediu desculpa pelo sucedido mas explicou que deveria devolver a calça com uma etiqueta própria e que me seria enviado o pack completo 
passado dois dias recebi o pack mas desta vez  das  calças veio com vários tamanhos abaixo do pedido 
quero que resolvam esta situação de uma vez por todas o meu trabalho não é compatível com o

In [19]:
# removing stop words
stop_words = set(stopwords.words('portuguese'))

print(stop_words)

{'estava', 'era', 'aos', 'seus', 'estejam', 'sejam', 'este', 'sou', 'estiver', 'eram', 'tivessem', 'à', 'seu', 'nossos', 'estejamos', 'o', 'uma', 'num', 'no', 'houverão', 'for', 'das', 'nossas', 'formos', 'é', 'pelo', 'estavam', 'estamos', 'nem', 'lhe', 'teríamos', 'hei', 'na', 'estive', 'estiveram', 'aquele', 'muito', 'estávamos', 'estivéramos', 'éramos', 'terei', 'estivemos', 'houvemos', 'sejamos', 'houveria', 'forem', 'com', 'eu', 'tivemos', 'estão', 'tive', 'ou', 'esta', 'pelas', 'esses', 'essas', 'houveríamos', 'por', 'dos', 'fomos', 'entre', 'estiverem', 'tivera', 'ela', 'houvesse', 'serei', 'tivéramos', 'quando', 'nosso', 'nas', 'houverem', 'lhes', 'tiver', 'suas', 'nós', 'qual', 'meus', 'está', 'a', 'teria', 'você', 'da', 'houveriam', 'terá', 'houvessem', 'de', 'houvera', 'tenha', 'houveremos', 'tinham', 'minha', 'ao', 'houve', 'já', 'isso', 'tivéssemos', 'em', 'seriam', 'eles', 'hajam', 'sem', 'houvéssemos', 'tivesse', 'serão', 'foi', 'fôssemos', 'estou', 'fosse', 'estivéssemo

In [21]:
tokenizedText = word_tokenize(textWOPontuation, language='portuguese')
print("List of words:\n",tokenizedText)

List of words:
 ['boa', 'noite', 'venho', 'mostrar', 'o', 'meu', 'total', 'descontentamento', 'e', 'frustação', 'relativamente', 'ao', 'serviço', 'prestado', 'fiz', 'uma', 'encomenda', 'de', 'várias', 'peças', 'uma', 'delas', 'um', 'pack', 'de', 'calças', 'de', 'fato', 'de', 'treino', 'por', '€', 'ao', 'chegar', 'a', 'encomenda', 'deparei-me', 'que', 'em', 'vez', 'do', 'pack', 'só', 'tinha', 'recebido', 'calça', 'em', 'vez', 'das', 'enviei', 'e-mail', 'para', 'o', 'apoio', 'ao', 'cliente', 'a', 'expor', 'a', 'situação', 'passado', 'algumas', 'horas', 'responderam', 'a', 'afirmar', 'que', 'teriam', 'entrado', 'em', 'contacto', 'comigo', 'várias', 'vezes', 'sem', 'sucesso', 'o', 'que', 'é', 'mentira', 'pois', 'confirmei', 'o', 'número', 'de', 'telefone', 'cedido', 'na', 'minha', 'conta', 'e', 'não', 'tinha', 'nenhuma', 'chamada', 'não', 'atendida', 'pediram', 'para', 'retribuir', 'a', 'chamada', 'contactei', 'então', 'por', 'chamada', 'telefonica', 'ao', 'qual', 'a', 'assistente', 'que',

In [22]:
cleanedText = []
for t in tokenizedText:
    if t not in stop_words:
        cleanedText.append(t)
print("Text without stopwords:\n",cleanedText)

Text without stopwords:
 ['boa', 'noite', 'venho', 'mostrar', 'total', 'descontentamento', 'frustação', 'relativamente', 'serviço', 'prestado', 'fiz', 'encomenda', 'várias', 'peças', 'pack', 'calças', 'fato', 'treino', '€', 'chegar', 'encomenda', 'deparei-me', 'vez', 'pack', 'recebido', 'calça', 'vez', 'enviei', 'e-mail', 'apoio', 'cliente', 'expor', 'situação', 'passado', 'algumas', 'horas', 'responderam', 'afirmar', 'entrado', 'contacto', 'comigo', 'várias', 'vezes', 'sucesso', 'mentira', 'pois', 'confirmei', 'número', 'telefone', 'cedido', 'conta', 'nenhuma', 'chamada', 'atendida', 'pediram', 'retribuir', 'chamada', 'contactei', 'então', 'chamada', 'telefonica', 'assistente', 'atendeu', 'nunca', 'pediu', 'desculpa', 'sucedido', 'explicou', 'deveria', 'devolver', 'calça', 'etiqueta', 'própria', 'enviado', 'pack', 'completo', 'passado', 'dois', 'dias', 'recebi', 'pack', 'desta', 'vez', 'calças', 'veio', 'vários', 'tamanhos', 'abaixo', 'pedido', 'quero', 'resolvam', 'situação', 'vez', 

In [25]:
stem = PorterStemmer()
stemmedText = []
for t in cleanedText:
    stemmedWord = stem.stem(t)
    stemmedText.append(stemmedWord)
print("Stemmed text :\n",stemmedText) 

Stemmed text :
 ['boa', 'noit', 'venho', 'mostrar', 'total', 'descontentamento', 'frustação', 'relativament', 'serviço', 'prestado', 'fiz', 'encomenda', 'vária', 'peça', 'pack', 'calça', 'fato', 'treino', '€', 'chegar', 'encomenda', 'deparei-m', 'vez', 'pack', 'recebido', 'calça', 'vez', 'enviei', 'e-mail', 'apoio', 'client', 'expor', 'situação', 'passado', 'alguma', 'hora', 'responderam', 'afirmar', 'entrado', 'contacto', 'comigo', 'vária', 'veze', 'sucesso', 'mentira', 'poi', 'confirmei', 'número', 'telefon', 'cedido', 'conta', 'nenhuma', 'chamada', 'atendida', 'pediram', 'retribuir', 'chamada', 'contactei', 'então', 'chamada', 'telefonica', 'assistent', 'atendeu', 'nunca', 'pediu', 'desculpa', 'sucedido', 'explic', 'deveria', 'devolv', 'calça', 'etiqueta', 'própria', 'enviado', 'pack', 'completo', 'passado', 'doi', 'dia', 'recebi', 'pack', 'desta', 'vez', 'calça', 'veio', 'vário', 'tamanho', 'abaixo', 'pedido', 'quero', 'resolvam', 'situação', 'vez', 'toda', 'trabalho', 'compatível'

In [27]:
newText = ''
for t in stemmedText:
  newText = newText + t + ' '
newText = newText.rstrip() # rstring (remove spaces at the right of the string)
print("Full sentence with changes so far:\n", newText)

Full sentence with changes so far:
 boa noit venho mostrar total descontentamento frustação relativament serviço prestado fiz encomenda vária peça pack calça fato treino € chegar encomenda deparei-m vez pack recebido calça vez enviei e-mail apoio client expor situação passado alguma hora responderam afirmar entrado contacto comigo vária veze sucesso mentira poi confirmei número telefon cedido conta nenhuma chamada atendida pediram retribuir chamada contactei então chamada telefonica assistent atendeu nunca pediu desculpa sucedido explic deveria devolv calça etiqueta própria enviado pack completo passado doi dia recebi pack desta vez calça veio vário tamanho abaixo pedido quero resolvam situação vez toda trabalho compatível ponto picket fazer devolução espero resolvam desagradável situação vez toda concertesa vou recomendar comprar nada marca
